## Reporter Agent

In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
from dotenv import load_dotenv

from agentic_newsroom.schemas.models import StoryBrief, ResearchPackage
from agentic_newsroom.utils.newsroom_logging import setup_logging

# Add src to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Load environment variables
env_path = project_root / '.env'
load_dotenv(dotenv_path=env_path)
setup_logging()

print("✓ Logging configured")
print("✓ Imports successful")
print(f"✓ Loading .env from: {env_path}")

✓ Logging configured
✓ Imports successful
✓ Loading .env from: /Users/juha/development/semantic-byte/ai-engineering/agentic-newsroom/.env


## Configuration

In [2]:
slug = "bounty-mutiny-1789-feature"
#slug = "emperor-penguins-life-on-thin-ice"
from agentic_newsroom.llm.openai import get_mini_model

mini_model = get_mini_model()
config = {"configurable": {"model": mini_model}}

story_brief = StoryBrief.load(slug=slug)
research_package = ResearchPackage.load(slug=slug)

reporter_state = {
    "story_brief": story_brief,
    "research_package": research_package
}

## Nodes


### Write Draft

In [ ]:
from agentic_newsroom.agents.reporter import write_draft
from IPython.display import display, Markdown

# Call the write_draft node
update1 = write_draft(reporter_state, config)
reporter_state.update(update1)


In [ ]:
draft_package = reporter_state["draft_package"]

draft_package_md = draft_package.to_markdown()
display(Markdown(draft_package_md))

#### Review Facts

In [ ]:
from agentic_newsroom.agents.reporter import review_facts

# Call the review_draft node
update2 = review_facts(reporter_state, config)
reporter_state.update(update2)

In [ ]:
# Display fact review
fact_review = reporter_state["fact_review"]
display(Markdown(fact_review.to_markdown()))



### Revise Facts

In [ ]:
from agentic_newsroom.agents.reporter import revise_facts

update3 = revise_facts(reporter_state, config)
reporter_state.update(update3)

In [ ]:
# Display the revised facts
revised_draft_package = reporter_state["draft_package"]
revised_draft_md = revised_draft_package.to_markdown()
display(Markdown(revised_draft_md))


### Review Style

In [ ]:
from agentic_newsroom.agents.reporter import review_style

# Call review_draft again for the second review
update4 = review_style(reporter_state, config)
reporter_state.update(update4)


In [ ]:
# Display the style review
style_review = reporter_state["style_review"]
display(Markdown(style_review.to_markdown()))


### Revise Style

In [ ]:
from agentic_newsroom.agents.reporter import revise_style
update5 = revise_style(reporter_state, config)
reporter_state.update(update5)

In [ ]:
# Display the draft after style revision
draft_package = reporter_state["draft_package"]
display(Markdown(draft_package.to_markdown()))

### Finalize Draft

In [ ]:
from agentic_newsroom.agents.reporter import finalize_draft
update6 = finalize_draft(reporter_state, config)
reporter_state.update(update6)

In [ ]:
# Display the draft after style revision
draft_package = reporter_state["draft_package"]
display(Markdown(draft_package.to_markdown()))

## Subgraph

In [3]:
from agentic_newsroom.agents.reporter import build_reporter_graph

graph = build_reporter_graph()

# Visualize the graph
print(graph.get_graph().draw_ascii())

  +-----------+  
  | __start__ |  
  +-----------+  
        *        
        *        
        *        
+-------------+  
| write_draft |  
+-------------+  
        *        
        *        
        *        
+--------------+ 
| review_facts | 
+--------------+ 
        *        
        *        
        *        
+--------------+ 
| revise_facts | 
+--------------+ 
        *        
        *        
        *        
  +---------+    
  | __end__ |    
  +---------+    


## Run the graph

In [ ]:
result = graph.invoke(reporter_state, config=config)

In [ ]:
print(result["draft_package"].full_draft)